## 4.2.6. MLP 규제
### 1) L2, L1 규제: 가중치 규제
- L2: 곡선을 부드럽게 만드는 제약 (weight_decay 옵션으로 부드러운 수준 조절)
    - 값이 클수롭 옵티마이저는 부드러운 모델 선택(L2 규제가 강해짐)
- L1: 희소한 솔루션을 만드는 데 사용 (대부분의 모델 파라미터가 0)

### 2) 드롭아웃: 구조적 규제
- 두 층 사이 유닛 연결을 랜덤하게 끊음
- --> 어떤 유닛도 다른 유닛에 전적으로 의존하지 못해서 모델 안정(+)
- <-> "상호 적응"(-): 다른 유닛 간 연결 희생하는 대신, 두 유닛 사이 연결이 지나치게 강해지는 상황--> 과대적합 (신경과학 용어)

드롭아웃을 적용한 MLP 구현
- 모델에 "드롭 확률 하이퍼파라미터" 추가
- 유닛 사이의 연결을 드롭할 확률 (보통 0.5)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultilayerPerceptron(nn.Module):
    def __int__(self, input_dim, hidden_dim, output_dim):
        super(MultilayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x_in, apply_softmax=False):
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(F.dropout(intermediate, p=0.5))
        
        if apply_softmax:
            output = F.softmax(output, dim=1)
            
        return output

## 4.3.2. 파이토치로 CNN 구현하기
### 인공 데이터와 Conv1d 클래스
- 특성 벡터 생성의 첫 단계: Conv1d 클래스 객체를 3차원 데이터 텐서에 적용
- 출력 텐서 크기가 줄어든다! 

In [5]:
batch_size = 2
one_hot_size = 10 # 어휘 사전의 크기(입력 채널 개수) : 증가하는 채널 차원의 크기 (10-->16-->32-->64)
sequence_width = 7 # 문자 시퀀스의 길이 : 줄어드는 출력텐서의 크기 (7-->5-->3-->1)

data = torch.randn(batch_size, one_hot_size, sequence_width) # (2,10,7)
conv1 = nn.Conv1d(in_channels=one_hot_size, out_channels=16, kernel_size=3)
# 입력 채널: 10개 --> 출력 채널: 16개 (늘어남!!!)
intermediate1 = conv1(data)

print(data.size())  # torch.Size([2, 10, 7])
# print(data)
print(intermediate1.size())  # torch.Size([2, 16, 5])
# print(intermediate1)

torch.Size([2, 10, 7])
torch.Size([2, 16, 5])


In [6]:
conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)

intermediate2 = conv2(intermediate1)
intermediate3 = conv3(intermediate2)

print(intermediate2.size())
print(intermediate3.size())

torch.Size([2, 32, 3])
torch.Size([2, 64, 1])


In [7]:
y_output = intermediate3.squeeze()
print(y_output.size())

torch.Size([2, 64])


In [8]:
# 특성 벡터를 줄이는 방법 2
print(intermediate1.view(batch_size, -1).size())

# 특성 벡터를 줄이는 방법 3
print(torch.mean(intermediate1, dim=2).size())

torch.Size([2, 80])
torch.Size([2, 16])


In [10]:
torch.mean(intermediate1, dim=2)

tensor([[ 0.2336,  0.0314,  0.5904, -0.2608,  0.3580, -0.1331,  0.0781, -0.0289,
          0.2339, -0.0439, -0.1255,  0.2845,  0.3261,  0.0895,  0.0185, -0.1021],
        [ 0.0596, -0.2360, -0.1638,  0.4077,  0.3807, -0.0178, -0.5495, -0.1608,
          0.0151, -0.3122, -0.0232, -0.3320,  0.4357,  0.0049, -0.1636, -0.3066]],
       grad_fn=<MeanBackward1>)

In [13]:
sum([ 0.3809,  0.9817,  0.0071,  0.6531, -0.8547]) / 5

0.23362000000000002

In [11]:
intermediate1

tensor([[[ 0.3809,  0.9817,  0.0071,  0.6531, -0.8547],
         [-0.1413, -0.1154, -0.2057, -0.4458,  1.0652],
         [ 0.3591,  0.7309,  1.3937, -0.4270,  0.8950],
         [-0.3762, -0.0295, -0.8151, -0.2099,  0.1269],
         [ 0.4933,  0.0622,  0.6837,  0.2406,  0.3100],
         [-0.2958,  0.7481, -1.0368, -0.4772,  0.3959],
         [-0.1905,  0.6143,  0.0034, -0.3635,  0.3267],
         [-0.6815,  0.4849,  0.2171, -0.2997,  0.1348],
         [ 0.4168,  0.2603,  0.3651, -0.2540,  0.3815],
         [ 0.0902,  0.8479, -0.2559, -0.4442, -0.4577],
         [-1.1323,  0.2508,  0.1969,  0.7232, -0.6659],
         [ 0.0829,  0.7409,  0.3315, -0.0563,  0.3238],
         [ 0.4004,  0.5064,  0.3465,  0.7065, -0.3291],
         [ 0.5829,  0.1141, -0.3796,  0.2355, -0.1053],
         [-0.8111,  0.5783,  0.3150, -0.0591,  0.0696],
         [-0.0143, -0.1024,  0.5793, -0.4656, -0.5074]],

        [[-0.2380, -0.3304,  0.2833,  0.4889,  0.0940],
         [-0.2316, -0.4655, -0.1956, -0.3711, 